# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:
#Bar Nahmuka
#205386899



## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
#Hebrew tokenizer installation:

#unmark if you want to use and need to install:
!pip install hebrew_tokenizer

In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\nahmu\assignment3-text_analysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

# Cleaning and dividing the data

First, we want clean data and the we need to divide the data into X and y.
So i built a cleaning function and i also built a tokinizer for later use:

In [11]:
#using the hebrew tokenizer
def tokenizer(hebrew_text):
    token=ht.tokenize(hebrew_text)
    tokens_list=list(filter(lambda x: (x[0]=='HEBREW'),token))
    tokens_list=[tuple[1] for tuple in tokens_list]
    tokens_list
    return tokens_list

In [12]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text) 
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [13]:
mapping = {'m': 0, 'f': 1}
df_train['gender'] = df_train['gender'].map(mapping)

X_train = df_train['story']
y_train = df_train['gender']
print('Training Data :', X_train.shape)
print('Training Data :', y_train.shape)

Training Data : (753,)
Training Data : (753,)


# Cross Validation and Grid Search

Now we need to vectorize the data.
I'm going to check both the CountVectorizer and TfidfVectorizer:

In [14]:
Countvec = CountVectorizer(max_df=0.9 ,min_df=5 ,max_features=10000, tokenizer=tokenizer)

X_train_Countvec = Countvec.fit_transform(X_train).toarray()
# Vectorizing the data with TfdifVectorizer
Tfidfvec = TfidfVectorizer(max_df=0.9 ,min_df=5 ,max_features=10000, tokenizer=tokenizer)

X_train_Tfidfvec = Tfidfvec.fit_transform(X_train).toarray()


Now i'm goinng to cross validate some models and see what their f1 score is:

In [15]:
clf_dict = {"Multinomial Naive Bayes":MultinomialNB(), "Gaussian Naive Bayes":GaussianNB(),
            "KNeighbors Classifier":KNeighborsClassifier(), "Decision Tree Classifier":DecisionTreeClassifier()}


print('using CountVectorizer:')
for key,value in clf_dict.items():
    scores = cross_val_score(value, X_train_Countvec, y_train, scoring='f1_macro', cv=5)
    print("{}: {}".format(key, round(scores.mean(), 3)))
    
print('\n')  
print('-------------------------------') 
print('\n') 

print('using TfidfVectorizer:')
for key,value in clf_dict.items():
    scores = cross_val_score(value, X_train_Tfidfvec, y_train, scoring='f1_macro', cv=5)
    print("{}: {}".format(key, round(scores.mean(), 3)))


using CountVectorizer:
Multinomial Naive Bayes: 0.669
Gaussian Naive Bayes: 0.459
KNeighbors Classifier: 0.537
Decision Tree Classifier: 0.56


-------------------------------


using TfidfVectorizer:
Multinomial Naive Bayes: 0.433
Gaussian Naive Bayes: 0.459
KNeighbors Classifier: 0.6
Decision Tree Classifier: 0.574


Now i'm going to fine tune the parameters to find the best combination for both Multinomial Naive Bayes model and CountVectorizer:

In [20]:
def best_pipeline(df_train):
    df_train['story'] = df_train['story'].apply(preprocess)

    pipeline = Pipeline([
        ('CountVectorizer', CountVectorizer(tokenizer=tokenizer)),
        ('Multinomial_Naive_Bayes', MultinomialNB())
    ])

    param = {
        'CountVectorizer__ngram_range': [(1, 1)],
        'CountVectorizer__min_df': [1, 5, 10],
        'CountVectorizer__max_df': [1.0, 0.7],
        'CountVectorizer__max_features': [1000, 5000],
        'CountVectorizer__tokenizer': [tokenizer],
        'Multinomial_Naive_Bayes__alpha': [0.01, 0.1, 0.5, 1.0],
        'Multinomial_Naive_Bayes__fit_prior': [True, False]
    }

    best_score = 0.0
    best_classifier = ''

    grid_search = GridSearchCV(pipeline, param_grid= param, scoring='f1_macro', cv=10, verbose=1)
    grid_search.fit(df_train['story'], df_train['gender'])

    best_f1_score = grid_search.best_score_
    best_params = grid_search.best_params_

    print(f"Best Multinomial Naive Bayes F1 Score: {best_f1_score:.4f}")
    print("Best Parameters:")
    for param, value in best_params.items():
        print(f"{param}: {value}")

    best_pipeline = grid_search.best_estimator_


In [21]:
best_pipeline(df_train)

Fitting 10 folds for each of 96 candidates, totalling 960 fits
Best Multinomial Naive Bayes F1 Score: 0.6997
Best Parameters:
CountVectorizer__max_df: 1.0
CountVectorizer__max_features: 5000
CountVectorizer__min_df: 10
CountVectorizer__ngram_range: (1, 1)
CountVectorizer__tokenizer: <function tokenizer at 0x0000026D51688310>
Multinomial_Naive_Bayes__alpha: 0.5
Multinomial_Naive_Bayes__fit_prior: False


0.6997

So the best parameters we found are:
for the Multinomial Naive Bayes model: alpha = 0.5, fit_prior = False
for the CountVectorizer: max_df=1.0, max_features=5000, min_df=10, ngram_range=(1, 1), tokenizer=tokenizer
Now lets cross validate the pipline with the best parameters found:

In [23]:
Best_pipeline = Pipeline([
    ('CountVectorizer', CountVectorizer(max_df=1.0, max_features=5000,min_df=10, ngram_range=(1, 1), tokenizer=tokenizer)),
    ('Multinomial_Naive_Bayes', MultinomialNB(alpha=0.5,fit_prior=False))
])

f1_scores = cross_val_score(Best_pipeline, X_train, y_train, scoring='f1_macro', cv=10)
mean_f1_score = f1_scores.mean()
print(f"Mean F1 Score: {mean_f1_score:.4f}")

Mean F1 Score: 0.6931


I got f1 score of 0.6931.

# Prediction

Lets try and predict the gender of the stories in the test set.

In [28]:
X_test = df_test['story'].apply(preprocess)
y_pred = Best_pipeline.fit(X_train, y_train).predict(X_test)
gender_mapping = {value: key for key, value in mapping.items()}  


inversed_y_pred = [gender_mapping[numeric_label] for numeric_label in y_pred]

df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': inversed_y_pred})

In [29]:
df_predicted.head(5)

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


In [30]:
Best_pipeline

Pipeline(steps=[('CountVectorizer',
                 CountVectorizer(max_features=5000, min_df=10,
                                 tokenizer=<function tokenizer at 0x0000026D51688310>)),
                ('Multinomial_Naive_Bayes',
                 MultinomialNB(alpha=0.5, fit_prior=False))])

In [31]:
df_predicted.tail(5)

,test_example_id,predicted_category
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [32]:
df_predicted.to_csv('classification_results.csv',index=False)